## 参加者の将来の歩行を予想するパーティクルフィルタの構築

本章では、てくペコ2015年度のデータを用い、以下の解析を行う。

* データの読み込み
* データの量子化
* 実施日時の天気の取得
* 実施日時の曜日の取得
* 量子化されたデータによる活動パターンのクラスタリング
* パーティクルフィルタの設計
* パーティクルフィルタによる予想結果の可視化


### データの読み込み

まずはデータベースに格納されたデータを読み込む

In [1]:
import os
import re
import datetime
import numpy
import pickle
from pprint import pformat
os.environ["DJANGO_SETTINGS_MODULE"] = "analyzer.settings"
import django
django.setup()

from analyzer import settings
from record.models import User, Group, Summary

username_expr = re.compile('(?P<season>[1-3])teku(?P<id>[0-9]+)')

datasets = {1: [], 2: [], 3: []}
seasons = {
    1: datetime.datetime.strptime('2015/09/11', '%Y/%m/%d'),
    2: datetime.datetime.strptime('2015/10/06', '%Y/%m/%d'),
    3: datetime.datetime.strptime('2015/11/01', '%Y/%m/%d')
}
for user in User.objects.all():
    m = username_expr.match(user.username)
    if not m:
        continue
    season = int(m.group('season'))
    season_start_date = seasons[season]
    date_dict = dict((season_start_date + datetime.timedelta(days=i), 0) for i in range(10))
    for summary in Summary.objects.filter(username=user.username):
        if summary.steps == 0: continue
        date_dict[summary.date] += summary.distance
    ary = [distance for date, distance in sorted(date_dict.items(), key=lambda x: x[0])]
    if sum(ary) == 0:
        continue
    datasets[season].append(ary)

for k, v in datasets.items():
    with open('10days_distance.season{}.pkl'.format(k), 'wb') as fp:
        pickle.dump(numpy.array(v), fp)

### データの量子化

よく行われるK-means法でのクラスタリングを用います。

In [11]:
import numpy
import scipy.cluster

data = numpy.array([])
for k, v in datasets.items():
    for row in v:
        data = numpy.hstack((data, row))

data = numpy.array(data).astype(numpy.float32)

print "data: ", data.shape
# データをクラスタリング
N = 8
codebook, destortion = scipy.cluster.vq.kmeans(data, N, iter=20, thresh=1e-05)
print codebook, destortion

# ベクトル量子化
# 各データをセントロイドに分類する
code, dist = scipy.cluster.vq.vq(data, codebook)

print "code: ", numpy.unique(code)

# 学習した結果を保存する
with open('vq_params.pkl', 'wb') as fp:
    pass


data:  (7040,)
[   794.41052246   8535.93554688  11618.77050781  33485.66796875
   2773.34448242   4657.61621094  17956.1171875    6445.45654297] 693.287
code:  [0 1 2 3 4 5 6 7]


### 実施日時の天気の取得

### 実施日時の曜日の取得